## 📦 Setup & Configuration

In [ ]:
# Включаем DEBUG логирование для всех компонентов
import os

os.environ['LOG_LEVEL'] = 'DEBUG'

from app.config import ensure_dotenv
from app.logging_config import configure_logging

# Загружаем .env
ensure_dotenv()
configure_logging()

print('✅ Logging configured')

✅ Logging configured



In [ ]:
# Проверяем все переменные окружения
env_vars = [
    'GIGACHAT_CREDENTIALS',
    'GIGACHAT_SCOPE',
    'GIGACHAT_MODEL',
    'EMBEDDINGS_MODEL',
    'LANGSMITH_TRACING',
    'LANGSMITH_API_KEY',
    'LANGSMITH_PROJECT',
]

print('🔧 Environment Variables:')
print('=' * 50)
for var in env_vars:
    val = os.getenv(var, 'NOT SET')
    if 'CREDENTIALS' in var or 'API_KEY' in var:
        val = f'{val[:20]}...' if len(val) > 20 else val
    print(f'{var}: {val}')

🔧 Environment Variables:
GIGACHAT_CREDENTIALS: MDE5YWNmM2YtMjFiOC03...
GIGACHAT_SCOPE: GIGACHAT_API_PERS
GIGACHAT_MODEL: GigaChat-2
EMBEDDINGS_MODEL: EmbeddingsGigaR
GIGACHAT_CREDENTIALS: MDE5YWNmM2YtMjFiOC03...
GIGACHAT_SCOPE: GIGACHAT_API_PERS
GIGACHAT_MODEL: GigaChat-2
EMBEDDINGS_MODEL: EmbeddingsGigaR
LANGSMITH_TRACING: true
LANGSMITH_API_KEY: lsv2_pt_a0adcfe85d3f...
LANGSMITH_PROJECT: city-assistant

LANGSMITH_TRACING: true
LANGSMITH_API_KEY: lsv2_pt_a0adcfe85d3f...
LANGSMITH_PROJECT: city-assistant


In [ ]:
# Загружаем RAG конфигурацию
import json

from app.rag.config import RAGConfig, get_rag_config

config = get_rag_config()

print('📋 RAG Configuration:')
print('=' * 50)
print('Chunking:')
print(f'  chunk_size: {config.chunking.chunk_size}')
print(f'  chunk_overlap: {config.chunking.chunk_overlap}')
print(f'  min_chunk_size: {config.chunking.min_chunk_size}')
print('\nSearch:')
print(f'  k: {config.search.k}')
print(f'  min_relevant: {config.search.min_relevant}')
print(f'  relevance_threshold: {config.search.relevance_threshold}')
print('\nIndex:')
print(f'  chroma_persist_dir: {config.index.chroma_persist_dir}')
print(f'  parsed_docs_dir: {config.index.parsed_docs_dir}')
print(f'  collection_name: {config.index.collection_name}')
print('\nRetriever:')
print(f'  vector_weight: {config.retriever.vector_weight}')
print(f'  bm25_weight: {config.retriever.bm25_weight}')

f:\dropped_out\prog.proj\sber_agent_hackathon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📋 RAG Configuration:
Chunking:
  chunk_size: 800
  chunk_overlap: 200
  min_chunk_size: 100
Chunking:
  chunk_size: 800
  chunk_overlap: 200
  min_chunk_size: 100

Search:
  k: 5
  min_relevant: 3
  relevance_threshold: 0.5

Index:
  chroma_persist_dir: data\chroma_db

Search:
  k: 5
  min_relevant: 3
  relevance_threshold: 0.5

Index:
  chroma_persist_dir: data\chroma_db
  parsed_docs_dir: data\parsed_docs
  collection_name: city_knowledge

Retriever:
  vector_weight: 0.5
  bm25_weight: 0.5

  parsed_docs_dir: data\parsed_docs
  collection_name: city_knowledge

Retriever:
  vector_weight: 0.5
  bm25_weight: 0.5


---
## 📄 Document Parsing

In [ ]:
# Проверяем существующие документы
from pathlib import Path

docs_path = config.index.parsed_docs_dir / 'all_documents.json'
print(f'📁 Documents path: {docs_path}')
print(f'   Exists: {docs_path.exists()}')

if docs_path.exists():
    size_kb = docs_path.stat().st_size / 1024
    print(f'   Size: {size_kb:.1f} KB')

In [ ]:
# Загружаем документы
from app.rag.indexer import load_parsed_documents

try:
    docs = load_parsed_documents()
    print(f'✅ Loaded {len(docs)} documents')
    print('\n📊 Documents by source type:')
    from collections import Counter

    source_counts = Counter(doc.source_type.value for doc in docs)
    for source, count in source_counts.items():
        print(f'   {source}: {count}')
except FileNotFoundError as e:
    print(f'❌ {e}')
    print('\n💡 Run: python -m app.rag.pipeline --max-services 10')

In [ ]:
# Пример документа
if 'docs' in dir() and docs:
    sample_doc = docs[0]
    print('📄 Sample Document:')
    print('=' * 50)
    print(f'Title: {sample_doc.title}')
    print(f'Doc ID: {sample_doc.doc_id}')
    print(f'URL: {sample_doc.url}')
    print(f'Source Type: {sample_doc.source_type.value}')
    print(f'Content Length: {len(sample_doc.content)} chars')
    print('\nContent Preview:')
    print('-' * 50)
    print(sample_doc.content[:500] + '...')

---
## 🗄️ Indexer Testing

In [ ]:
# Создаём indexer
from app.rag.indexer import HybridIndexer

indexer = HybridIndexer()
print('✅ HybridIndexer created')

In [ ]:
# Проверяем/создаём индекс
result = indexer.ensure_indexed()
print('\n📊 Index Status:')
print(f'   ensure_indexed(): {result}')
print(f'   BM25 docs: {len(indexer._bm25_docs)}')

In [ ]:
# Метаданные индекса
metadata = indexer.load_metadata()
print('📋 Index Metadata:')
print(json.dumps(metadata, indent=2, ensure_ascii=False))

---
## 🔎 Retriever Testing

In [ ]:
# Получаем retriever
retriever = indexer.get_retriever()
print(f'✅ Retriever type: {type(retriever).__name__}')

In [ ]:
# Sample queries для тестирования
SAMPLE_QUERIES = [
    'Как получить СНИЛС для ребёнка?',
    'Какие документы нужны для регистрации по месту жительства?',
    'Как оформить пенсию?',
    'Как получить загранпаспорт?',
    'Какие льготы положены многодетным семьям?',
]

print('📝 Sample Queries:')
for i, q in enumerate(SAMPLE_QUERIES, 1):
    print(f'   {i}. {q}')

In [ ]:
# Тестируем поиск
query = SAMPLE_QUERIES[0]
print(f'🔍 Query: "{query}"')
print('=' * 50)

results = indexer.search(query, k=5)

print(f'\n📊 Found {len(results)} documents:')
for i, doc in enumerate(results, 1):
    title = doc.metadata.get('title', 'N/A')
    url = doc.metadata.get('url', 'N/A')
    print(f'\n{i}. {title}')
    print(f'   URL: {url}')
    print(f'   Content: {doc.page_content[:200]}...')

In [ ]:
# Сравнение BM25 vs Vector vs Ensemble
from langchain_community.retrievers import BM25Retriever

query = SAMPLE_QUERIES[1]
print(f'🔍 Query: "{query}"')
print('=' * 60)

# BM25 only
bm25_results = indexer._bm25_retriever.invoke(query)[:3]
print('\n📗 BM25 Results:')
for i, doc in enumerate(bm25_results, 1):
    print(f'   {i}. {doc.metadata.get("title", "N/A")[:50]}')

# Ensemble
ensemble_results = indexer.search(query, k=3)
print('\n📘 Ensemble (BM25 + Vector) Results:')
for i, doc in enumerate(ensemble_results, 1):
    print(f'   {i}. {doc.metadata.get("title", "N/A")[:50]}')

---
## 🔄 RAG Graph Testing

In [ ]:
# Импортируем RAG graph
from app.rag.graph import RAGState, rag_graph

print('✅ RAG Graph loaded')
print(f'   Nodes: {list(rag_graph.nodes.keys())}')

In [ ]:
# Тестируем RAG graph
query = 'Как оформить загранпаспорт?'

initial_state: RAGState = {
    'query': query,
    'k': 5,
    'min_relevant': 3,
    'is_toxic': False,
    'toxicity_response': None,
    'rewritten_query': None,
    'retrieved_docs': [],
    'deduplicated_docs': [],
    'graded_docs': [],
    'metadata': {},
}

print(f'🔍 Running RAG Graph with query: "{query}"')
print('=' * 60)

# Запускаем граф
result = rag_graph.invoke(initial_state)

print('\n📊 Result:')
print(f'   is_toxic: {result.get("is_toxic")}')
print(f'   rewritten_query: {result.get("rewritten_query")}')
print(f'   retrieved_docs: {len(result.get("retrieved_docs", []))}')
print(f'   graded_docs: {len(result.get("graded_docs", []))}')
print(f'\n📋 Metadata:')
print(json.dumps(result.get('metadata', {}), indent=2, ensure_ascii=False))

In [ ]:
# Тестируем токсичный запрос
toxic_query = 'блин, нихера не работает'

toxic_state: RAGState = {
    'query': toxic_query,
    'k': 5,
    'min_relevant': 3,
    'is_toxic': False,
    'toxicity_response': None,
    'rewritten_query': None,
    'retrieved_docs': [],
    'deduplicated_docs': [],
    'graded_docs': [],
    'metadata': {},
}

print(f'🔍 Testing toxicity with: "{toxic_query}"')
print('=' * 60)

result = rag_graph.invoke(toxic_state)

print('\n📊 Result:')
print(f'   is_toxic: {result.get("is_toxic")}')
print(f'   toxicity_response: {result.get("toxicity_response")}')

---
## 🧪 Custom Config Testing

Примеры разных конфигураций для тестирования

In [ ]:
# Sample configs для разных сценариев
from app.rag.config import ChunkingConfig, IndexConfig, RAGConfig, RetrieverConfig, SearchConfig

SAMPLE_CONFIGS = {
    'default': RAGConfig(),
    'small_chunks': RAGConfig(chunking=ChunkingConfig(chunk_size=400, chunk_overlap=100)),
    'large_chunks': RAGConfig(chunking=ChunkingConfig(chunk_size=1600, chunk_overlap=400)),
    'more_results': RAGConfig(search=SearchConfig(k=10, min_relevant=5)),
    'bm25_heavy': RAGConfig(retriever=RetrieverConfig(vector_weight=0.3, bm25_weight=0.7)),
    'vector_heavy': RAGConfig(retriever=RetrieverConfig(vector_weight=0.7, bm25_weight=0.3)),
}

print('📋 Available Sample Configs:')
for name, cfg in SAMPLE_CONFIGS.items():
    print(f'\n{name}:')
    print(f'   chunk_size={cfg.chunking.chunk_size}, k={cfg.search.k}')
    print(
        f'   vector_weight={cfg.retriever.vector_weight}, bm25_weight={cfg.retriever.bm25_weight}'
    )

In [ ]:
# Тестирование с кастомным конфигом

# from app.rag.config import set_rag_config, reset_rag_config
#
# # Применяем кастомный конфиг
# set_rag_config(SAMPLE_CONFIGS['bm25_heavy'])
#
# # Создаём новый indexer с этим конфигом
# custom_indexer = HybridIndexer()
# custom_indexer.ensure_indexed()
#
# # Тестируем
# results = custom_indexer.search('как получить паспорт', k=3)
# for doc in results:
#     print(doc.metadata.get('title'))
#
# # Возвращаем дефолтный конфиг
# reset_rag_config()

---
## 📊 Batch Testing

In [ ]:
# Batch тестирование всех sample queries
print('📊 Batch Search Results')
print('=' * 60)

for query in SAMPLE_QUERIES:
    results = indexer.search(query, k=3)
    print(f'\n🔍 "{query}"')
    for i, doc in enumerate(results, 1):
        title = doc.metadata.get('title', 'N/A')[:40]
        print(f'   {i}. {title}')